In [1]:
import shapefile
from shapely.geometry import shape, Point, mapping
import numpy as np
import pandas as pd

base = "../../data/raw/"
filename = "2016_LGA_SHAPE/LGA_2016_AUST"

sf = shapefile.Reader(base+filename)

nswshapes = list(map(lambda x: shape(x.__geo_interface__), sf.shapes()[:130]))
centroids = list(map(lambda x: shape(x.__geo_interface__).centroid, sf.shapes()[:130]))

In [2]:
import pandas as pd
df = pd.DataFrame(sf.records())
nswdf = df[df[4] =="New South Wales"]
# drop "No usual address" - not a useful LGA
nswdf = nswdf[nswdf[0] != "LGA19499"]
nswdf.columns = ["code", "codenum", "name", "unknown", "state", "unknown2"]
nswdf["clean_name"] = nswdf["name"].str.extract('([^\\(]*)', expand=False).str.strip()
nswdf



,code,codenum,name,unknown,state,unknown2,clean_name
0,LGA10050,10050,Albury (C),1,New South Wales,305.9459,Albury
1,LGA10130,10130,Armidale Regional (A),1,New South Wales,8620.6990,Armidale Regional
2,LGA10250,10250,Ballina (A),1,New South Wales,484.9389,Ballina
3,LGA10300,10300,Balranald (A),1,New South Wales,21690.6753,Balranald
4,LGA10470,10470,Bathurst Regional (A),1,New South Wales,3817.8646,Bathurst Regional
5,LGA10550,10550,Bega Valley (A),1,New South Wales,6278.8811,Bega Valley
6,LGA10600,10600,Bellingen (A),1,New South Wales,1600.4337,Bellingen
7,LGA10650,10650,Berrigan (A),1,New South Wales,2065.7759,Berrigan
8,LGA10750,10750,Blacktown (C),1,New South Wales,240.0487,Blacktown
9,LGA10800,10800,Bland (A),1,New South Wales,8557.6535,Bland


In [6]:
import json
lga_json = list(map(lambda x: x.__geo_interface__, sf.shapes()[:130]))

geojson = {'type': 'FeatureCollection',
            'features': [{'type':'Feature', 'geometry':x, 'properties': {'name':nswdf.iloc[i]['clean_name']}} for i, x in enumerate(lga_json)]
          }


#with open('geojson.json','w') as f:
#    json.dump(geojson, f)
#geojson[:5]

In [8]:
#target = pd.read_csv(base+"cleaned_target.csv")

#set(nswdf["clean_name"]).difference(set(target["LGA"]))
#set(target["LGA"]).difference(set(nswdf["clean_name"]))

In [9]:
import os

tweetdf =pd.read_csv(os.path.join(base, "tweet.csv"),header=0)



In [10]:
loctweetdf = tweetdf.loc[tweetdf['lat'].notnull(),:]

In [ ]:
loctweetdf

In [11]:
def whichlga(tweetpoints, lgashapes):
    #loop over 130 NSW LGAs and return a name or "None"
    #NB you get names from Census data which is 2016 - pre amalgamations
    #test by whether in bounding box. If in two, go to shapes. If still in two, one with smaller bounding box
    output = []
    nums = []
    amalgamations = {'Botany Bay': 'Bayside',
                     'Rockdale'  : 'Bayside',
                     'Gundagai'  : 'Cootamundra-Gundagai',
                     'Western Plains Regional' : 'Dubbo Regional',
                     'Unincorporated NSW' : 'Unincorporated Far West'
                    }
    
    for point in tweetpoints:
        found = []
        distances = []
        for i, nswshape in enumerate(nswshapes):
            if point.within(nswshape):
                #import pdb; pdb.set_trace()
                found.append(i)
                break
            else:
                distances.append(point.distance(nswshape))
                
        #print(len(found))
        nums.append(len(found))
        if len(found) == 1:
            clean_name = nswdf.iloc[found[0]].clean_name
        else:
            #out on the water
            clean_name = nswdf.iloc[np.argmin(np.asarray(distances))].clean_name
            
        if clean_name in amalgamations:
            clean_name = amalgamations[clean_name]
        output.append(clean_name)
            
    return pd.Series(output), nums
            

In [12]:
tweetpoints = []

loctweetdf.apply(lambda x: tweetpoints.append(shape(Point(x['lng'], x['lat']))), axis=1)


1         None
4         None
10        None
11        None
17        None
18        None
30        None
40        None
46        None
50        None
54        None
57        None
58        None
72        None
73        None
74        None
75        None
76        None
78        None
84        None
90        None
98        None
105       None
112       None
154       None
162       None
164       None
165       None
167       None
185       None
          ... 
522271    None
522282    None
522293    None
522294    None
522296    None
522310    None
522314    None
522330    None
522337    None
522345    None
522356    None
522375    None
522383    None
522384    None
522385    None
522386    None
522387    None
522388    None
522389    None
522391    None
522399    None
522409    None
522411    None
522416    None
522417    None
522420    None
522422    None
522434    None
522438    None
522439    None
Length: 75082, dtype: object

In [20]:

output, found = whichlga(tweetpoints, nswshapes)
found = np.asarray(found)
np.histogram(found)

(array([15725,     0,     0,     0,     0,     0,     0,     0,     0,
        59357]),
 array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]))

In [21]:
#tweetdf[pd.isnull(output)]
tweetdf.loc[tweetdf['lat'].notnull(),'lga'] = np.asarray(output)


In [22]:
tweetdf.to_csv("tweets_w_lga.csv")

In [24]:
tweetdf = pd.read_csv("tweets_w_lga.csv")
loctweetdf = tweetdf.loc[~pd.isnull(tweetdf['lat'])]

In [25]:
aggdf = loctweetdf[['lat','lng','text','lga']].groupby(['lat','lng', 'lga']).agg('count').reset_index()
#output


In [31]:
tweetdf.loc[(tweetdf['lng'] > 151.237) & (tweetdf['lng'] < 151.24) & (tweetdf['lat'] > -33.9) & (tweetdf['lat'] < -33.8)]

,Unnamed: 0,Unnamed: 0.1,id,time,created_at,from_user_name,text,filter_level,possibly_sensitive,withheld_copyright,...,from_user_timezone,from_user_lang,from_user_tweetcount,from_user_followercount,from_user_friendcount,from_user_favourites_count,from_user_listed,from_user_withheld_scope,from_user_created_at,lga
73,73,73,951303433503387649,1515643387,2018-01-11 04:03:07,PairsonnalitesA,ASIA | AUNZ — Maori teens pressured for nude ...,low,0.0,NaN,...,Sydney,en-gb,1128837,3201,1444,0,154,NaN,2010-05-29 17:26:03,Woollahra
74,74,74,951303435437010946,1515643388,2018-01-11 04:03:08,PairsonnalitesA,ASIA | AUNZ — Court puts fishhooks in taonga ...,low,0.0,NaN,...,Sydney,en-gb,1128838,3201,1444,0,154,NaN,2010-05-29 17:26:03,Woollahra
75,75,75,951303437148286976,1515643388,2018-01-11 04:03:08,PairsonnalitesA,ASIA | AUNZ — Maori artist brings Australia's...,low,0.0,NaN,...,Sydney,en-gb,1128839,3201,1444,0,154,NaN,2010-05-29 17:26:03,Woollahra
76,76,76,951303438477926401,1515643388,2018-01-11 04:03:08,PairsonnalitesA,ASIA | AUNZ — Maori representation and social...,low,0.0,NaN,...,Sydney,en-gb,1128840,3201,1444,0,154,NaN,2010-05-29 17:26:03,Woollahra
425,425,425,951318546012712960,1515646990,2018-01-11 05:03:10,PairsonnalitesA,ASIA | AUNZ — Family mourns as Akubra Amy suc...,low,0.0,NaN,...,Sydney,en-gb,1128867,3201,1444,0,154,NaN,2010-05-29 17:26:03,Woollahra
426,426,426,951318548248342529,1515646991,2018-01-11 05:03:11,PairsonnalitesA,ASIA | AUNZ — Australia should learn to respe...,low,0.0,NaN,...,Sydney,en-gb,1128868,3201,1444,0,154,NaN,2010-05-29 17:26:03,Woollahra
427,427,427,951318550769123328,1515646991,2018-01-11 05:03:11,PairsonnalitesA,ASIA | AUNZ — They have anger management issu...,low,0.0,NaN,...,Sydney,en-gb,1128869,3201,1444,0,154,NaN,2010-05-29 17:26:03,Woollahra
428,428,428,951318552170061824,1515646992,2018-01-11 05:03:12,PairsonnalitesA,ASIA | AUNZ — Everyone hates me' but Allah: H...,low,0.0,NaN,...,Sydney,en-gb,1128870,3201,1444,0,154,NaN,2010-05-29 17:26:03,Woollahra
830,830,830,951334124198928385,1515650704,2018-01-11 06:05:04,PairsonnalitesA,ASIA | AUNZ — Emanuel makes history with same...,low,0.0,NaN,...,Sydney,en-gb,1128888,3201,1444,0,154,NaN,2010-05-29 17:26:03,Woollahra
831,831,831,951334125931122688,1515650705,2018-01-11 06:05:05,PairsonnalitesA,ASIA | AUNZ — Trans Teenagers Will Get Their ...,low,0.0,NaN,...,Sydney,en-gb,1128889,3201,1444,0,154,NaN,2010-05-29 17:26:03,Woollahra


In [32]:
from emoji import UNICODE_EMOJI
set(UNICODE_EMOJI)

ModuleNotFoundError: No module named 'emoji'

In [26]:
import dash
import dash_core_components as dcc
from plotly.graph_objs import *

app = dash.Dash()

colorscale = [
    'rgb(193, 193, 193)',
    'rgb(239,239,239)',
    'rgb(195, 196, 222)',
    'rgb(144,148,194)',
    'rgb(101,104,168)',
    'rgb(65, 53, 132)'
]

#lga boundaries
lga_layers = list(map(lambda x: { 
                'id' : x['properties']['name'],
                'sourcetype' : "geojson",
                'source': x,
                'type'   : 'line',
                'layout': {
                    'line-join': 'round',
                    'line-cap' : 'round'
                },
                'color': '#888',
                'line': dict(width=0.5),
                }, geojson['features']))

#lga labels
labels = dict(type='scattermapbox',
            lon=[x.x for x in centroids],
            lat=[x.y for x in centroids],
            mode='text',
            text=list(nswdf.clean_name),
            #hoverinfo='text'
             )

data = [dict(
        type = 'scattermapbox',
        lon = aggdf['lng'],
        lat = aggdf['lat'],
        text = aggdf['lga'],
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'circle',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = colorscale,
            cmin = 0,
            color = aggdf['text'],
            cmax = aggdf['text'].max(),
            colorbar=dict(
                title="Number of Tweets"
            )
        ))]

data.append(labels)

app.layout = dcc.Graph(
    id = "mapbox",
    
    figure={
        "data": data,
        "layout": dict(
            hovermode = "closest",
            margin = dict(l = 0, r = 0, t = 0, b = 0),
            mapbox = dict(
                layers = lga_layers,
                accesstoken = "pk.eyJ1IjoiZGVzdWx0aXIiLCJhIjoiY2pmZDN4M2FlMTEzYjJ3bWlyc2VhaDkxaiJ9.VvezwDLTQlh2GVAEsauwlg",
                bearing = 0,
                center = dict(lat=-32.162833, lon=147.031917),
                style = "light",
                pitch = 0,
                zoom = 6.0,
            )
        )
    },
    style = {"height": 750}
)

app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Apr/2018 13:32:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2018 13:32:24] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2018 13:32:24] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2018 13:32:24] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2018 15:17:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2018 15:17:34] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2018 15:17:34] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2018 15:17:34] "GET /favicon.ico HTTP/1.1" 200 -


In [ ]:
tweetdf['time'].min()